## 第 5 章 误差反向传播法

### 5.1 计算图: 
*计算图将计算过程用图形表示出来, 这里说的图形是数据结构图，通过多个节点和边表示*
- 5.1.1 **用计算图求解**: 从左向右进行计算, 为正向传播(forward propagation); 从右向左, 反向传播(backward propagation), 导数计算
- 5.1.2 **局部计算**: 与自己相关的某个小范围, 无论全局发生了什么，都能只根据与自己相关的信息输出接下来的结果。
- 5.1.3 **为何用计算图解题**: 通过反向传播高效计算导数。(局部计算 + 利用计算图可以将中间的计算结果全部保存起来)

### 5.2 链式法则: 
*将局部导数向正方向的反方向（从右到左）传递, 传递这个局部导数的原理，是基于链式法则(chain rule)的。*
- 5.2.1 **计算图的反向传播**: 将信号 E 乘以节点的局部导数$(\frac{∂y}{∂x})$，然后将结果传递给下一个节点。假设 $y=f(x)=x^2$, 则局部导数为$(\frac{∂y}{∂x})$=2x. 把这个局部导数乘以上游传过来的值(本例中为 E)，然后传递给前面的节点。这就是反向传播的计算顺序。通过这样的计算，可以高效地求出导数的值，这是反向传播的要点。
- 5.2.2 **什么是链式法则**: 从复合函数说起, 复合函数是由多个函数构成的函数。比如，$z = (x + y)^2$ 是由 $z = t^2$, $t = x + y$ 两个式子构成的。如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。$(\frac{∂z}{∂x} = \frac{∂z}{∂t} · \frac{∂t}{∂x})$ ∂t 正好可以互相抵消。$(\frac{∂z}{∂t} = 2t, \frac{∂t}{∂x} = 1, 2t · 1 = 2(x + y)$。
- 5.2.3 **链式法则和计算图**: 计算图的反向传播从右到左传播信号。反向传播的计算顺序是，先将节点的输入信号乘以节点的局部导数（偏导数），然后再传递给下一个节点。

### 5.3 反向传播
- 5.3.1 **加法节点的反向传播**: $z = x + y$ 加法节点的反向传播只是将输入信号输出到下一个节点。
- 5.3.2 **乘法节点的反向传播**: $z = x · y$ 乘法的反向传播会将上游的值乘以正向传播时的输入信号的“翻转值”后传递给下游。
- 5.3.3 **苹果的例子**